# GETTING STARTED WITH SQL AND BIG QUERY

# Introduction

Structured Query Language, or SQL, is the programming language used with databases, and it is an important skill for any data scientist. In this course, you'll build your SQL skills using BigQuery, a web service that lets you apply SQL to huge datasets.

In this lesson, you'll learn the basics of accessing and examining BigQuery datasets. After you have a handle on these basics, we'll come back to build your SQL skills.

## YOUR FIRST BIG QUERY COMMANDS

In this we will import the bigquery python package

In [1]:
from google.cloud import bigquery

The first step in the workflow is to create a Client object. As you'll soon see, this Client object will play a central role in retrieving information from BigQuery datasets.

In [2]:
# create a client object
client = bigquery.Client()



Using Kaggle's public dataset BigQuery integration.

We'll work with a dataset of posts on Hacker News, a website focusing on computer science and cybersecurity news.

In BigQuery, each dataset is contained in a corresponding project. In this case, our hacker_news dataset is contained in the bigquery-public-data project. To access the dataset,

    We begin by constructing a reference to the dataset with the dataset() method.
    Next, we use the get_dataset() method, along with the reference we just constructed, to fetch the dataset.



In [3]:
# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [4]:
 #List all the tables in the "hacker_news" dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset (there are four!)
for table in tables:  
    print(table.table_id)

full


Similar to how we fetched a dataset, we can fetch a table. In the code cell below, we fetch the full table in the hacker_news dataset.

In [5]:
# Construct a reference to the "full" table
table_ref = dataset_ref.table("full")

# API request - fetch the table
table = client.get_table(table_ref)

In the next section, you'll explore the contents of this table in more detail. For now, take the time to use the image below to consolidate what you've learned so far.

#  Table schema

The structure of a table is called its schema. We need to understand a table's schema to effectively pull out the data we want.

In this example, we'll investigate the full table that we fetched above.


In [6]:
# Print all information on the columns in the full 
table.schema

[SchemaField('title', 'STRING', 'NULLABLE', None, 'Story title', (), None),
 SchemaField('url', 'STRING', 'NULLABLE', None, 'Story url', (), None),
 SchemaField('text', 'STRING', 'NULLABLE', None, 'Story or comment text', (), None),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', None, 'Is dead?', (), None),
 SchemaField('by', 'STRING', 'NULLABLE', None, "The username of the item's author.", (), None),
 SchemaField('score', 'INTEGER', 'NULLABLE', None, 'Story score', (), None),
 SchemaField('time', 'INTEGER', 'NULLABLE', None, 'Unix time', (), None),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', None, 'Timestamp for the unix time', (), None),
 SchemaField('type', 'STRING', 'NULLABLE', None, 'Type of details (comment, comment_ranking, poll, story, job, pollopt)', (), None),
 SchemaField('id', 'INTEGER', 'NULLABLE', None, "The item's unique id.", (), None),
 SchemaField('parent', 'INTEGER', 'NULLABLE', None, 'Parent comment ID', (), None),
 SchemaField('descendants', 'INTEGER', 'NULLABL

Each SchemaField tells us about a specific column (which we also refer to as a field). In order, the information is:

    The name of the column
    The field type (or datatype) in the column
    The mode of the column ('NULLABLE' means that a column allows NULL values, and is the default)
    A description of the data in that column



In [7]:
# Preview the first five lines of the "full" table
client.list_rows(table, max_results=5).to_dataframe()


,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,"I would rather just have wired earbuds, period...",<NA>,zeveb,<NA>,1591717736,2020-06-09 15:48:56+00:00,comment,23467666,23456782,<NA>,<NA>,<NA>
1,None,None,DNS?,<NA>,nly,<NA>,1572810465,2019-11-03 19:47:45+00:00,comment,21436112,21435130,<NA>,<NA>,<NA>
2,None,None,These benchmarks seem pretty good. Filterable...,<NA>,mrkeen,<NA>,1591717727,2020-06-09 15:48:47+00:00,comment,23467665,23467426,<NA>,<NA>,<NA>
3,None,None,Oh really?<p>* Excel alone uses 86.1MB of priv...,<NA>,oceanswave,<NA>,1462987532,2016-05-11 17:25:32+00:00,comment,11677248,11676886,<NA>,<NA>,<NA>
4,None,None,These systems are useless. Of the many flaws:...,<NA>,nyxxie,<NA>,1572810473,2019-11-03 19:47:53+00:00,comment,21436113,21435025,<NA>,<NA>,<NA>


# The list_rows() method will also let us look at just the information in a specific column. If we want to see the first five entries in the by column, for example, we can do that!

# Preview the first five entries in the "by" column of the "full" table
client.list_rows(table, selected_fields=table.schema[:1], max_results=5).to_dataframe()


# SELECT FROM AND WHERE IN SQL

## Introduction

Now that you know how to access and examine a dataset, you're ready to write your first SQL query! As you'll soon see, SQL queries will help you sort through a massive dataset, to retrieve only the information that you need.

We'll begin by using the keywords SELECT, FROM, and WHERE to get data from specific columns based on conditions you specify.

For clarity, we'll work with a small imaginary dataset pet_records which contains just one table, called pet

SELECT ... FROM

The most basic SQL query selects a single column from a single table. To do this,

    specify the column you want after the word SELECT, and then
    specify the table after the word FROM.


WHERE ...

BigQuery datasets are large, so you'll usually want to return only the rows meeting specific conditions. You can do this using the WHERE clause.

Example: What are all the U.S. cities in the OpenAQ dataset?

Now that you've got the basics down, let's work through an example with a real dataset. We'll use an OpenAQ dataset about air quality.

In [11]:
from google.cloud import bigquery

client = bigquery.Client()

dataset_ref = client.dataset("openaq",project="bigquery-public-data")

dataset = client.get_dataset(dataset_ref)

tables = list(client.list_tables(dataset))

for table in tables:
    print(table.table_id)

global_air_quality




Using Kaggle's public dataset BigQuery integration.
global_air_quality

The dataset contains only one table, called global_air_quality. We'll fetch the table and take a peek at the first few rows to see what sort of data it contains

In [14]:
# Construct a reference to the global_air_quality table
table_ref = dataset_ref.table("global_air_quality")

table = client.get_table(table_ref)
client.list_rows(table,max_results=5).to_dataframe()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours,location_geom
0,"Borówiec, ul. Drapałka",Borówiec,PL,bc,0.85217,2022-04-28 07:00:00+00:00,µg/m³,GIOS,1.0,52.276794,17.074114,POINT(52.276794 1)
1,"Kraków, ul. Bulwarowa",Kraków,PL,bc,0.91284,2022-04-27 23:00:00+00:00,µg/m³,GIOS,1.0,50.069308,20.053492,POINT(50.069308 1)
2,"Płock, ul. Reja",Płock,PL,bc,1.41000,2022-03-30 04:00:00+00:00,µg/m³,GIOS,1.0,52.550938,19.709791,POINT(52.550938 1)
3,"Elbląg, ul. Bażyńskiego",Elbląg,PL,bc,0.33607,2022-05-03 13:00:00+00:00,µg/m³,GIOS,1.0,54.167847,19.410942,POINT(54.167847 1)
4,"Piastów, ul. Pułaskiego",Piastów,PL,bc,0.51000,2022-05-11 05:00:00+00:00,µg/m³,GIOS,1.0,52.191728,20.837489,POINT(52.191728 1)


Everything looks good! So, let's put together a query. Say we want to select all the values from the city column that are in rows where the country column is 'US' (for "United States").

In [19]:
query = """
        SELECT city
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """



Take the time now to ensure that this query lines up with what you learned above.
Submitting the query to the dataset

We're ready to use this query to get information from the OpenAQ dataset. As in the previous tutorial, the first step is to create a Client object.


In [16]:
# create a client object
client = bigquery.Client()





Using Kaggle's public dataset BigQuery integration.

We begin by setting up the query with the query() method. We run the method with the default parameters, but this method also allows us to specify more complicated settings that you can read about in the documentation. We'll revisit this later.


In [20]:
#set up the query
query_job = client.query(query)

In [21]:
# Next we will run the query and convert the results to a pandas DataFrame

us_cities = query_job.to_dataframe()

In [22]:
us_cities.city.value_counts().head()

city
Phoenix-Mesa-Scottsdale                     39414
Los Angeles-Long Beach-Santa Ana            27479
Riverside-San Bernardino-Ontario            26887
New York-Northern New Jersey-Long Island    25417
San Francisco-Oakland-Fremont               22710
Name: count, dtype: int64

In [23]:
# Want to run multiple queries? 

query = """

SELECT city, country FROM `bigquery-public-data.openaq.global_air_quality` WHERE country='US';
"""

You can select all columns with *

In [24]:
# Want to run multiple queries? 

query = """

SELECT * FROM `bigquery-public-data.openaq.global_air_quality` WHERE country='US';
"""

Question: What's up with the triple quotation marks (""")?

Answer: These tell Python that everything inside them is a single string, even though we have line breaks in it. The line breaks aren't necessary, but they make it easier to read your query.
Question: Do you need to capitalize SELECT and FROM?

Answer: No, SQL doesn't care about capitalization. However, it's customary to capitalize your SQL commands, and it makes your queries a bit easier to read.

# Group By, Having & Count

Introduction

Now that you can select raw data, you're ready to learn how to group your data and count things within those groups. This can help you answer questions like:

    How many of each kind of fruit has our store sold?
    How many species of animal has the vet office treated?

To do this, you'll learn about three new techniques: GROUP BY, HAVING and COUNT(). Once again, we'll use this made-up table of information on pets.

COUNT()

COUNT(), as you may have guessed from the name, returns a count of things. If you pass it the name of a column, it will return the number of entries in that column.

COUNT() is an example of an aggregate function, which takes many values and returns one. (Other examples of aggregate functions include SUM(), AVG(), MIN(), and MAX().) As you'll notice in the picture above, aggregate functions introduce strange column names (like f0__). Later in this tutorial, you'll learn how to change the name to something more descriptive.

GROUP BY

GROUP BY takes the name of one or more columns, and treats all rows with the same value in that column as a single group when you apply aggregate functions like COUNT().

For example, say we want to know how many of each type of animal we have in the pets table. We can use GROUP BY to group together rows that have the same value in the Animal column, while using COUNT() to find out how many ID's we have in each group.

GROUP BY ... HAVING

HAVING is used in combination with GROUP BY to ignore groups that don't meet certain criteria.

So this query, for example, will only include groups that have more than one ID in them.

Example: Which Hacker News comments generated the most discussion?

Ready to see an example on a real dataset? The Hacker News dataset contains information on stories and comments from the Hacker News social networking site.

We'll work with the comments table and begin by printing the first few rows

In [39]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "comments" table
table_ref = dataset_ref.table("full")

# API request - fetch the table
table = client.get_table(table_ref)

client.list_rows(table, max_results=5).to_dataframe()


,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,"I would rather just have wired earbuds, period...",<NA>,zeveb,<NA>,1591717736,2020-06-09 15:48:56+00:00,comment,23467666,23456782,<NA>,<NA>,<NA>
1,None,None,DNS?,<NA>,nly,<NA>,1572810465,2019-11-03 19:47:45+00:00,comment,21436112,21435130,<NA>,<NA>,<NA>
2,None,None,These benchmarks seem pretty good. Filterable...,<NA>,mrkeen,<NA>,1591717727,2020-06-09 15:48:47+00:00,comment,23467665,23467426,<NA>,<NA>,<NA>
3,None,None,Oh really?<p>* Excel alone uses 86.1MB of priv...,<NA>,oceanswave,<NA>,1462987532,2016-05-11 17:25:32+00:00,comment,11677248,11676886,<NA>,<NA>,<NA>
4,None,None,These systems are useless. Of the many flaws:...,<NA>,nyxxie,<NA>,1572810473,2019-11-03 19:47:53+00:00,comment,21436113,21435025,<NA>,<NA>,<NA>


# There is an issue with this DataSet



Let's use the table to see which comments generated the most replies. Since:

    the parent column indicates the comment that was replied to, and
    the id column has the unique ID used to identify each comment,

we can GROUP BY the parent column and COUNT() the id column in order to figure out the number of comments that were made as responses to a specific comment. (This might not make sense immediately -- take your time here to ensure that everything is clear!)

Furthermore, since we're only interested in popular comments, we'll look at comments with more than ten replies. So, we'll only return groups HAVING more than ten ID's.


In [41]:
# Query to select comments that received more than 10 replies
query_popular = """
                SELECT parent, COUNT(id)
                FROM `bigquery-public-data.hacker_news.full`
                GROUP BY parent
                HAVING COUNT(id) > 10
                """


In [42]:
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query_popular, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
popular_comments = query_job.to_dataframe()

# Print the first five rows of the DataFrame
popular_comments.head()




,parent,f0_
0,19503094,176
1,32448825,126
2,16428624,53
3,25783152,661
4,12986759,98


liasing and other improvements

A couple hints to make your queries even better:

    The column resulting from COUNT(id) was called f0__. That's not a very descriptive name. You can change the name by adding AS NumPosts after you specify the aggregation. This is called aliasing, and it will be covered in more detail in an upcoming lesson.
    If you are ever unsure what to put inside the COUNT() function, you can do COUNT(1) to count the rows in each group. Most people find it especially readable, because we know it's not focusing on other columns. It also scans less data than if supplied column names (making it faster and using less of your data access quota).

Using these tricks, we can rewrite our query:

In [44]:
# Improved version of earlier query, now with aliasing & improved readability
query_improved = """
                 SELECT parent, COUNT(1) AS NumPosts
                 FROM `bigquery-public-data.hacker_news.full`
                 GROUP BY parent
                 HAVING COUNT(1) > 10
                 """

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query_improved, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
improved_df = query_job.to_dataframe()

# Print the first five rows of the DataFrame
improved_df.head()

,parent,NumPosts
0,13529792,67
1,31743110,81
2,32875846,68
3,29082014,57
4,22326931,46


Note on using GROUP BY

Note that because it tells SQL how to apply aggregate functions (like COUNT()), it doesn't make sense to use GROUP BY without an aggregate function. Similarly, if you have any GROUP BY clause, then all variables must be passed to either a

    GROUP BY command, or
    an aggregation function.

Consider the query below:

In [45]:
query_good = """
             SELECT parent, COUNT(id)
             FROM `bigquery-public-data.hacker_news.full`
             GROUP BY parent
             """